In [2]:
!pip install tensorflow tensorflow-datasets transformers datasets evaluate

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np

print("Libraries imported successfully!")

Libraries imported successfully!


In [4]:
print("\nLoading IMDB dataset...")

imdb_dataset = load_dataset("imdb")
print("Dataset loaded successfully!")
print("\nDataset structure:")
print(imdb_dataset)
print("\nExample of a training review:")
print(imdb_dataset["train"][0])


Loading IMDB dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset loaded successfully!

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Example of a training review:
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vie

In [5]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english" # A good starting point
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_imdb = imdb_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [15]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

Device set to use cpu


In [16]:
review = "This movie was absolutely fantastic! The acting was superb, and the plot was engaging."
result = classifier(review)
print(result)

review = "This movie was terrible. The plot was boring and the acting was awful."
result = classifier(review)
print(result)

[{'label': 'POSITIVE', 'score': 0.9998832941055298}]
[{'label': 'NEGATIVE', 'score': 0.9998061060905457}]


In [20]:
import os
os.environ["WANDB_DISABLED"] = "true"

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Example usage (after fine-tuning, you'll have predictions and labels)
# metric.compute(predictions=predictions, references=labels)

print("--- Evaluating the model on the test set ---")
trainer = Trainer(model=model)
small_eval_dataset = tokenized_imdb["test"].shuffle(seed=42).select(range(1000))

predictions = trainer.predict(
    test_dataset=small_eval_dataset,
    metric_key_prefix="eval"
)

final_metrics = compute_metrics((predictions.predictions, predictions.label_ids))
print("\nFinal evaluation metrics:")
print(final_metrics)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


--- Evaluating the model on the test set ---



Final evaluation metrics:
{'accuracy': 0.881}
